# Búsqueda simple

En esta prática vamos aprender a buscar elemento simple utilizando expresiones booleanas. Vamos a utilizar Elastic Search como si fuera una base de datos. Por último entenderemos como podemos utilizar elastic search como motor de agregación.

## Introducción

Este primer bloque de código sirve para configurar el Notebook

In [ ]:
from IPython.display import JSON

Ahora vamos a descargar el cliente de ElasticSearch en Python.

In [ ]:
pip install elasticsearch==7.10.1

Por último, creamos la conexion con el servidor de Elastic Search desplegado

In [ ]:
from elasticsearch import Elasticsearch
es = Elasticsearch(
    ['elasticsearch']
)
JSON(es.info())

## Importando los datos

En primer lugar vamos a descargar los datos usando el comando:

In [ ]:
!wget "https://gist.githubusercontent.com/aagea/204e6803c0c9705b07aadceab9f7bd1e/raw/2dcdef641ab29411705f1d00e2fb1396c2101e93/hotels.json"

In [ ]:
es.indices.delete(index="hotels",ignore=[400,404])
!curl -H "Content-Type: application/json" -XPOST "http://elasticsearch:9200/hotels/_bulk?pretty" --data-binary "@hotels.json" >> /dev/null;

Ahora vamos a recuperar a los 10 primeros hoteles que hemos indexado.

In [ ]:
JSON(es.search(index="hotels"))

Por otro lado vamos a revisar el mapping type que se ha generado.

In [ ]:
JSON(es.indices.get_mapping(index="hotels"))

## Busquedas simples

Ahora vamos buscar en los documentos que hemos almacenado, lo primero que vamos a hacer es buscar la palabra `Saerim`en los nombres de los hoteles.

In [ ]:
request_body={
  "query": {
    "match": {
      "name": "Saerim"
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

Como podeis ver el filtro que se usa es textual y no requiere que el nombre sea exactamente como el documento guardardo, para hacer esto debemos usar la busqueda term.

In [ ]:
request_body={
  "query": {
    "term": {
      "name": "Saerim Hotel"
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

Sin embargo esta búsqueda no devuelve ningún resultado debido a que el elemento esta almacenado como tipo text. Sin embargo, en el mapping type se ha añadido un tipo keyword que almacena la misma información como una keyword. Por tanto si hacemos la busqueda llamando a este campo el sistema devuelve los resultados correctos.

In [ ]:
request_body={
  "query": {
    "term": {
      "name.keyword": "Saerim Hotel"
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

## Term level queries

Las term leve queries son consultas de filtrado es decir no analizan el texto generado sino que devuelven los resultado utlizando los terminos exactos.

### Term
El campo debe contener el valor exacto de la query.

In [ ]:
request_body={
  "query": {
    "term": {
      "name.keyword": {
          "value":"saerim Hotel",
          "case_insensitive":"true"
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

### Range

Devuelve los documentos dentro de un rango.

In [ ]:
request_body={
  "query": {
    "range": {
      "stars": {
          "lte":2
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

### Prefix
Devuelve todos los documentos que empiezan por una consulta concreta.

In [ ]:
request_body={
  "query": {
    "prefix": {
      "name.keyword": {
          "value":"sae",
          "case_insensitive":"true"
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

## Fuzzy
Permite devolver documentos que tiene valores similares a la consulta dada, se utiliza la distancia de Levenshtein para saber si el termino es próximo.

In [ ]:
request_body={
  "query": {
    "fuzzy": {
      "name.keyword": {
          "value":"Sarim Hotel",
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

## Ejercicio 1 

Busca los hoteles que tengan entre 3 o cuatro estrellas.

Busca los hoteles que tenga más de cuatro cientas habitaciónes.

Encuentra el hotel Ding Seoul Hotel... creo que se llamaba así...

Encuentra todos los hoteles que no tengan internet.

## Consultas compuestas

Para componer queries la forma sencilla es utilizar la boolean query que permite definir  utilizando cuatro estructuras que queremos encontrar:

* La estructura **must** indica que los documentos deber hacer matchin con los filtros indicados en la query.
* La estructura **should** indica filtros que son opcionales y que aumetan la puntuacion de los resultados.
* La estructura **mustnot** elimina todos los resultados que cumplan las condiciones indicadas. No afecta al scoring.
* La escructura **filter** devuelve todos los resultados que cumplan las condiciones indicadas. No afecta al scoring.

Los boolean query se pueden anidar si requieren consultas aun más complejas.

In [ ]:
request_body={
  "query": {
    "bool": {
      "must": {
          "range": {
              "rooms":{
                  "gte":"300"
              }
          }
      },
      "should": [
          {
            "range": {
              "price":{
                "lte":"250"
              }
            }
          },
          {
            "range": {
              "stars":{
                  "gte":"5"
              }
            }
          }
      ],
      "filter": {
          "range": {
              "stars":{
                  "gte":"3"
              }
          }
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

Con la opcion `"minimum_should_match" : 1`podemos obligar a que se cumpla al menos una condición.

In [ ]:
request_body={
  "query": {
    "bool": {
      "must": {
          "range": {
              "rooms":{
                  "gte":"300"
              }
          }
      },
      "should": [
          {
            "range": {
              "price":{
                "lte":"250"
              }
            }
          },
          {
            "range": {
              "stars":{
                  "gte":"5"
              }
            }
          }
      ],
      "filter": {
          "range": {
              "stars":{
                  "gte":"3"
              }
          }
      },
      "minimum_should_match" : 1
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

## Ejercicio 2

Busca hoteles que tengan menos de 3 estrellas y que valgan menos de 100 o tengan menos de habitaciones.

Entre los hoteles de que tengan internet, busca los que tenga menos 150 habitaciones. Estaría genial que tuviera cafetería (`Coffee shop`).

Busca todos los hoteles que su nombre empieza por `Hotel`pero no tienen internet.

## Aggregations

Las agregaciones nos permite calcular metricas de los documentos selecionados, de esta forma podemos conocer más detalles de nuestro dataset.

Un agregación básica nos permitirá conocer el precio de medio de los hoteles de nuestro dataset.

In [ ]:
request_body={
  "aggs": {"avg_price" : { "avg" : { "field" : "price" } }}
}
JSON(es.search(index="hotels", body=request_body))

Podemos hacer agregaciones de elementos filtrados, por ejemplo solo los hoteles de cinco estrellas.

In [ ]:
request_body={
  "aggs": {"avg_price" : { "avg" : { "field" : "price" } }},
  "query": {"range": {"stars": {"gte":"5"}}}
}
JSON(es.search(index="hotels", body=request_body))

Tambien se pueden extraer estadisticas de un campo concreto.

In [ ]:
request_body={
  "aggs": {"stats_rooms" : { "stats" : { "field" : "rooms" } }}
}
JSON(es.search(index="hotels", body=request_body))

Aunque una funcionalidad aún más interesantes es la capacida de crear buckets.

In [ ]:

request_body={
  "aggs" : {
    "hotels" : {
      "filters" : {
        "filters" : {
          "spas" :   { "match" : { "service" : "spa"   }},
          "weddings" : { "match" : { "service" : "wedding" }}
        }
      }
    }
  }
}
JSON(es.search(index="hotels", body=request_body))

A la vez se puede realizar agregaciones de cada uno de los buckets creados.

In [ ]:
request_body={
  "aggs" : {
    "hotels" : {
      "filters" : {
        "filters" : {
          "spas" :   { "match" : { "service" : "spa"   }},
          "weddings" : { "match" : { "service" : "wedding" }}
        }
      },
      "aggs": {
          "avg_price":{"avg":{"field":"price"}}
      }
    }
    
  }
}
JSON(es.search(index="hotels", body=request_body))

## Ejercicio 3

Calcula el numero habitaciones media que tiene los hoteles de los siguientes rangos: <150, entre 150 y 250, entre 250 y 350, y mayore de 350. Ver [documetación de range](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/search-aggregations-bucket-range-aggregation.html) 

Calcula el menor precio de un hotel con un servicio concreto. Ver [documentación de term](https://www.elastic.co/guide/en/elasticsearch/reference/7.17/search-aggregations-bucket-terms-aggregation.html)